In [1]:
import sys
import time
from PIL import Image, ImageDraw
import utils
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import datasets, transforms
from torch.autograd import Variable
%matplotlib inline
import numpy as np
import dataset
import random
import math
import json
from region_loss import RegionLoss
from models import *
import h5py
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.cuda.manual_seed(int(time.time()))

In [2]:
transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean = [ 0.5, 0.5, 0.5 ],std = [ 0.25, 0.25, 0.25 ]),
                       ])

In [3]:
a = 'FullChannels()'

In [4]:
model = eval(a)

In [5]:
load_net('backup/bestFullChannels().weights', model)
model.cuda()

FullChannels(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3)
      (1): ReLU(inplace)
      (2): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
      (3): ReLU(inplace)
    )
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
      (1): ReLU(inplace)
      (2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
      (3): ReLU(inplace)
    )
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
      (1): ReLU(inplace)
      (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      (3): ReLU(inplace)
    )
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Sequential(
      (0): Conv

In [6]:
count = 0
for k, v in model.state_dict().items():
    data=v.cpu().numpy()
    if count<16 and count%2==0:
        minimum=1.0/2**7
    else:
        minimum=1.0/2**7
    for x in np.nditer(data, op_flags=['readwrite']):
        if x[...]>1:
            x[...]=0.99
        if x[...]<-1:
            x[...]=-0.99
        x[...]=round(x[...]/minimum)*minimum
    param = torch.from_numpy(data)  
    v.copy_(param)

In [7]:
with open('val_list.txt', 'r') as outfile:
    lines = json.load(outfile)

In [8]:
test_infos = lines[0]

In [9]:
pred_infos = []
anchors     = model.anchors
num_anchors = model.num_anchors
#anchor_step = len(anchors)/num_anchors
anchor_step = int(len(anchors)/num_anchors)
proposals = 0.0
total = 0.0
for i in range(len(test_infos)):
    imgpath = test_infos[i][0]
    box = test_infos[i][1]
    label = np.zeros(4)
    label[0:2] = box[2:4]
    label[2:4] = box[0:2]
    img = Image.open(imgpath).convert('RGB')
    img = img.resize((model.width, model.height))
    timg = transform(img)
    timg = timg.view(1, 3, model.height, model.width)
    output = model(timg.cuda()).data
    batch = output.size(0)
    h = output.size(2)
    w = output.size(3)
    output = output.view(batch*num_anchors, 5, h*w).transpose(0,1).contiguous().view(5, batch*num_anchors*h*w)
    grid_x = torch.linspace(0, w-1, w).repeat(h,1).repeat(batch*num_anchors, 1, 1).view(batch*num_anchors*h*w).cuda()
    grid_y = torch.linspace(0, h-1, h).repeat(w,1).t().repeat(batch*num_anchors, 1, 1).view(batch*num_anchors*h*w).cuda()
    xs = torch.sigmoid(output[0]) + grid_x
    ys = torch.sigmoid(output[1]) + grid_y

    anchor_w = torch.Tensor(anchors).view(num_anchors, anchor_step).index_select(1, torch.LongTensor([0]))
    anchor_h = torch.Tensor(anchors).view(num_anchors, anchor_step).index_select(1, torch.LongTensor([1]))
    anchor_w = anchor_w.repeat(batch, 1).repeat(1, 1, h*w).view(batch*num_anchors*h*w).cuda()
    anchor_h = anchor_h.repeat(batch, 1).repeat(1, 1, h*w).view(batch*num_anchors*h*w).cuda()
    ws = torch.exp(output[2]) * anchor_w
    hs = torch.exp(output[3]) * anchor_h
    det_confs = torch.sigmoid(output[4])
    sz_hw = h*w
    sz_hwa = sz_hw*num_anchors
    det_confs = convert2cpu(det_confs)
    xs = convert2cpu(xs)
    ys = convert2cpu(ys)
    ws = convert2cpu(ws)
    hs = convert2cpu(hs)        

    for b in range(batch):
        det_confs_inb = det_confs[b*sz_hwa:(b+1)*sz_hwa].numpy()
        xs_inb = xs[b*sz_hwa:(b+1)*sz_hwa].numpy()
        ys_inb = ys[b*sz_hwa:(b+1)*sz_hwa].numpy()
        ws_inb = ws[b*sz_hwa:(b+1)*sz_hwa].numpy()
        hs_inb = hs[b*sz_hwa:(b+1)*sz_hwa].numpy()      
        ind = np.argmax(det_confs_inb)

        bcx = xs_inb[ind]
        bcy = ys_inb[ind]
        bw = ws_inb[ind]
        bh = hs_inb[ind]

        box = [bcx/w, bcy/h, bw/w, bh/h]

        iou = utils.bbox_iou(box, label, x1y1x2y2=False)
        print(iou)
        
        pred_infos.append([imgpath,box])
        proposals = proposals + iou
        total = total+1

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
tensor(0.0357)
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
tensor(0.0123)
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
tensor(0.0084)
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
tensor(0.0632)
0.0
0.0
0.0
0.0
0.0
tensor(0.0003)
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [10]:
proposals/total

tensor(0.0007)

In [11]:
with open('pred_infos.json', 'w') as outfile:
    json.dump(pred_infos, outfile)

In [12]:
param_list=[]
for key, value in model.state_dict().items():
    flat_weight = value.contiguous().view(value.numel())
    param_list.extend(flat_weight.tolist())
for i in param_list:
    i = float(i)

In [13]:
len(param_list)

48414

In [14]:
import struct
fp = open("param_v19best.bin",'wb')
s = struct.pack('f'*len(param_list), *param_list)
fp.write(s)

193656

In [15]:
img.size

(500, 333)